In [ ]:
input_img = ("")

In [3]:
import cv2
import numpy as np

# Load image
img = cv2.imread("ky.jpg")  # BGR format in OpenCV

# Split channels
b, g, r = cv2.split(img)

# Define threshold for red detection
threshold = 100  # tweak as needed

# Create binary mask: 1 if red is strong and green/blue are weak
binary_map = np.where((r > threshold) & (g < threshold) & (b < threshold), 1, 0)

# Optional: save as image
cv2.imwrite("ky_binary_map.png", (binary_map * 255).astype(np.uint8))


True

In [ ]:
import rasterio
from rasterio.features import shapes
import numpy as np
from skimage import measure, morphology
from shapely.geometry import shape, mapping
import fiona
import json

# --------------------------
# Parameters
# --------------------------
input_raster = "ky_binary_map.png"  # binary or probability raster
output_geojson = "ky_buildings.geojson"
threshold = 0.5                     # if probability raster
simplify_tolerance = 2.0            # units in arbitrary coordinate system
min_area_pixels = 20                 # remove small noise

# --------------------------
# Load raster
# --------------------------
with rasterio.open(input_raster) as src:
    raster = src.read(1)

# --------------------------
# Arbitrary transform
# --------------------------
# Let's assume top-left corner at (1000, 1000), each pixel = 1 unit
from rasterio.transform import from_origin
transform = from_origin(1000, 1000, 1, 1)

# --------------------------
# Threshold if raster is soft probability
# --------------------------
mask = raster > threshold

# --------------------------
# Remove small objects (noise)
# --------------------------
mask = morphology.remove_small_objects(mask.astype(bool), min_size=min_area_pixels)

# --------------------------
# Label connected components
# --------------------------
labels = measure.label(mask)

# --------------------------
# Polygonize each connected component
# --------------------------
polygons = []
for region in measure.regionprops(labels):
    coords = region.coords
    single_mask = np.zeros_like(mask, dtype=np.uint8)
    single_mask[tuple(coords.T)] = 1
    for geom, val in shapes(single_mask, mask=single_mask, transform=transform):
        if val == 1:
            poly = shape(geom)
            poly = poly.simplify(simplify_tolerance, preserve_topology=True)
            polygons.append(poly)

# --------------------------
# Remove empty geometries
# --------------------------
polygons = [poly for poly in polygons if poly.area > 0]

# --------------------------
# Save to GeoJSON
# --------------------------
geojson_features = []
for idx, poly in enumerate(polygons):
    geojson_features.append({
        "type": "Feature",
        "geometry": mapping(poly),
        "properties": {"id": idx}
    })

geojson_dict = {
    "type": "FeatureCollection",
    "features": geojson_features
}

with open(output_geojson, "w") as f:
    json.dump(geojson_dict, f)

print(f"Polygonized buildings saved to {output_geojson}")
